## <font color=green> Tratamento de dados: indexação, junção, concatenação e combinação

- Em muitas aplicações, os dados podem estar espalhados em vários arquivos ou bancos de dados, ou podem estar organizados em um formato que não seja fácil de analisar

In [1]:
import pandas as pd
import numpy as np

### Indexação Hierárquica

- A indexação hierárquica é um recurso que permite ter vários níveis de índices (dois ou mais) em um eixo.

In [2]:
data = pd.Series(np.random.rand(9),
                    index=[['a','a','a','b','b','c','c','d','d'],
                           [1,2,3,1,3,1,2,2,3]])
data

a  1    0.623578
   2    0.811677
   3    0.649938
b  1    0.841282
   3    0.883429
c  1    0.548181
   2    0.506593
d  2    0.695759
   3    0.982488
dtype: float64

- O que você está vendo é um visão elegante de um Series com um MultiIndex como índice. As “lacunas” na exibição do índice significam “utilize o rótulo imediatamente anterior”:

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

- Com um objeto hierarquicamente indexado, a chamada indexação parcial é possível, permitindo selecionar subconjuntos dos dados de forma concisa

In [4]:
data['b']

1    0.841282
3    0.883429
dtype: float64

In [5]:
data.loc[['b','d']]

b  1    0.841282
   3    0.883429
d  2    0.695759
   3    0.982488
dtype: float64

- A seleção é até mesmo possível a partir de um nível “mais interno”

In [6]:
data.loc[:,2]

a    0.811677
c    0.506593
d    0.695759
dtype: float64

- A indexação hierárquica desempenha um papel na reformatação dos dados e nas operações baseadas em grupos, como compor uma tabela pivô.
- Podemos reorganizar, por exemplo, os dados em um DataFrame usando o seu método unstack.

In [7]:
data.unstack()

,1,2,3
a,0.623578,0.811677,0.649938
b,0.841282,NaN,0.883429
c,0.548181,0.506593,NaN
d,NaN,0.695759,0.982488


- Em um DataFrame, qualquer eixo pode ter um índice hierárquico

In [8]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)), 
                     index=[['a','a','b','b'],[1,2,1,2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

- Os níveis hierárquicos podem ter nomes (como strings ou quaisquer objetos Python).

In [9]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [10]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### Reorganizando e ordenando níveis

- Ocasionalmente precisaremos reorganizar a ordem dos níveis em um eixo ou ordenar os dados de acordo com os valores em um nível específico.
- swaplevel aceita dois números ou nomes de níveis e devolve um novo objetos com os níveis trocados (os dados, porém, permanecem inalterados)

In [11]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

- sort_index, por outro lado, ordena os dados usando os valores de um só nível. Ao trocar níveis, não é incomum usar sort_index também para que o resultado seja ordenado em ordem lexicográfica de acordo com o nível indicado

In [12]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [13]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Estatísticas de resumo por níveis

- Muitas estatísticas descritivas ou de resumo em DataFrame e em Series têm uma opção level; com ela, podemos especificar o nível de acordo com o qual queremos fazer uma agregação, em um eixo em particular.

In [14]:
frame.groupby(level='key2').sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [15]:
frame.T.groupby(level='color').sum().T


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexando com as colunas de um DataFrame

In [16]:
frame = pd.DataFrame({'a': range(7), 
                      'b': range(7, 0, -1), 
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


- A função set_index de DataFrame cria um novo DataFrame usando uma ou mais colunas índice de linha

In [17]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

- Por padrão, as colunas são removidas do DataFrame, embora possam ser mantidas.

In [18]:
frame.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

- reset_index, por outro lado, faz o inverso de set_index; os níveis dos índices hierárquicos são passados para as colunas

In [19]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### Junções no DataFrame no estilo de bancos de dados

- Operações de merge ou de join combinam conjuntos de dados associando linhas por meio de uma ou mais chaves em bancos de dados relacionais
- A função merge do pandas é o ponto de entrada principal para usar essesw algoritmos em seus dados

In [20]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 
                    'data2': range(3)})

In [21]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [22]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


- Fazendo junção de muitos-para-um
- Os dados em df1 têm várias linhas de rótulos a e b, enquanto df2 tem apenas uma linha para cada valor na coluna key

In [23]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


- Observe que a coluna não foi especificada para fazer a junção. A função merge utilizou como chaves os nomes das colunas que se sobrepõem

- No entanto, é possível especificá-la explicitamente com o parâmetro on

In [24]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [25]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 
                    'data2': range(3)})

In [26]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [27]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [28]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


- No nosso exemplo os valores 'c' e 'd' e os dados associados estão ausentes no resultado.
- É importante destacar que por padrão, merge executa uma junção do tipo 'inner'.

- Outras opções possíveis são: 'left', 'right' e 'outer'

In [29]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,a,2.0,0.0
1,a,4.0,0.0
2,a,5.0,0.0
3,b,0.0,1.0
4,b,1.0,1.0
5,b,6.0,1.0
6,c,3.0,NaN
7,d,NaN,2.0


- Fazendo junção de muitos-para-muitos

In [30]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 
                    'data1': range(6)})

df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

In [31]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [32]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [33]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [34]:
pd.merge(df1, df2, on='key', how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


- Para um merge com vários chaves, passe uma lista de nomes de coluna

In [35]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 
                     'key2': ['one', 'two', 'one'],
                     'lval': [1,2,3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 
                      'key2': ['one', 'one', 'one', 'two'], 
                      'rval': [4, 5, 6, 7]})

In [36]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [37]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [38]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,bar,one,3.0,6.0
1,bar,two,NaN,7.0
2,foo,one,1.0,4.0
3,foo,one,1.0,5.0
4,foo,two,2.0,NaN


- merge tem uma opção suffixes para especificar strings a serem concatenadas nos nomes que se sobrepõem, nos objetos DataFrame à esquerda e à direita

In [39]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [40]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Fazendo merge com base no índice

- Você pode passar left_index=True ou right_index=True (ou ambos) para informar que o índice deverá ser usado como a chave do merge

In [41]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 
                      'value': range(6)})

right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [42]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [43]:
right1

,group_val
a,3.5
b,7.0


In [44]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [45]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


- O DataFrane tem uma instância conveniente de join para o merge pelo índice, que pode ser usada para combinar vários objetos DataFrame com índices iguais ou semelhantes, porém com colunas que não se sobrepõem

In [46]:
left1.join(right1, on='key', how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


### Concatenando ao longo de um eixo

- A função concat do pandas oferece uma forma consistente de tratar a concatenação de dados

In [47]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [48]:
s1

a    0
b    1
dtype: int64

In [49]:
s2

c    2
d    3
e    4
dtype: int64

In [50]:
s3

f    5
g    6
dtype: int64

In [51]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

- Por padrão, concat atua em axis=0, gerando outra Series.
- Se você passar axis=1, o resultado será um DataFrame.

In [52]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


- Não há sobreposição no outro eixo, o que gera a união ordenada (a junção 'outer') dos índices

- Podemos fazer a sua intersecção passando join='inner'

In [53]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [54]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [55]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


- Os rótulos 'f' e 'g' desapareceram por causa da opção join='inner'

- Suponha que quiséssemos criar um índice hieráquico no eixo da concatenação. Para isso, utilize o argumeto keys

In [56]:
s1

a    0
b    1
dtype: int64

In [57]:
s3

f    5
g    6
dtype: int64

In [58]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [59]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


- No caso da combinação de Series ao longo de axis=1, as keys passam a ser os cabeçalhos das colunas do DataFrame

In [60]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


- A mesma lógica se estende aos objetos DataFrame

In [61]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [62]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index=['a', 'c'], columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [63]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

- Quando o índice das linhas dos DataFrames não contém nenhum dado relevante, podemos passar ignore_index=True

In [64]:
df1 = pd.DataFrame(np.random.randn(3,4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2,3), columns=['b', 'd', 'a'])

In [65]:
df1

,a,b,c,d
0,0.295998,-0.033232,0.646685,0.997042
1,-0.232241,-0.419263,-0.575633,1.256072
2,-2.743113,-0.484675,0.973348,-1.001014


In [66]:
df2

,b,d,a
0,-1.829708,-0.140332,-0.795058
1,0.102632,-0.267755,-1.289729


In [67]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.295998,-0.033232,0.646685,0.997042
1,-0.232241,-0.419263,-0.575633,1.256072
2,-2.743113,-0.484675,0.973348,-1.001014
3,-0.795058,-1.829708,NaN,-0.140332
4,-1.289729,0.102632,NaN,-0.267755


### Combinando dados com sobreposição

- Podemos ter dois conjuntos de dados cujos índices se sobreponham de forma total ou parcial. Nesse caso, considere a função where do Numpy, que executa o equivalente a uma expressão if-else, porém orientado a arrays.

In [68]:
a = pd.Series([np.nan, 2.5, 0, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series([0, np.nan, 2, np.nan, np.nan, 5], index=['a', 'b', 'c', 'd', 'e', 'f'])

In [69]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [70]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [71]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

- Com  DataFrames, combine_first faz o mesmo coluna a coluna

In [72]:
df1 = pd.DataFrame({'a': [1, np.nan, 5, np.nan], 
                    'b': [np.nan, 2, np.nan, 6], 
                    'c': range(2,18,4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [73]:
df2 = pd.DataFrame({'a': [5, 4, np.nan, 3, 7], 
                    'b': [np.nan, 3, 4, 6, 8]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [74]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### Reformatando um DataFrame

- Podemos formatar um DataFrame fazendo com que valores se tornem coluna e vice-versa

In [75]:
df_longo = pd.DataFrame({
        "estudante":
            ["André", "Bernardo", "Cristina", "Debóra",
             "André", "Bernardo", "Cristina", "Debóra",
             "André", "Bernardo", "Cristina", "Debóra"],
        "escola":
            ["Z", "Y", "Z", "Y",
             "Z", "Y", "Z", "Y",
             "Z", "Y", "Z", "Y"],
        "matéria":
            ["inglês", "inglês", "inglês", "inglês",
             "matemática", "matemática", "matemática", "matemática",
             "física", "física", "física", "física"],
        "nota":
            [10, 100, 1000, 10000,
             20, 200, 2000, 20000,
             50, 500, 5000, 50000]
})

df_longo

,estudante,escola,matéria,nota
0,André,Z,inglês,10
1,Bernardo,Y,inglês,100
2,Cristina,Z,inglês,1000
3,Debóra,Y,inglês,10000
4,André,Z,matemática,20
5,Bernardo,Y,matemática,200
6,Cristina,Z,matemática,2000
7,Debóra,Y,matemática,20000
8,André,Z,física,50
9,Bernardo,Y,física,500


- Tornando o DataFrame menos longo e mais amplo

In [76]:
df_longo.pivot_table(index=["estudante", "escola"], 
                    columns='matéria', 
                    values='nota')

,matéria,física,inglês,matemática
estudante,escola,,,
André,Z,50.0,10.0,20.0
Bernardo,Y,500.0,100.0,200.0
Cristina,Z,5000.0,1000.0,2000.0
Debóra,Y,50000.0,10000.0,20000.0


- Adicionando margens para com a soma dos valores por linhas e por colunas

In [77]:
df_longo.pivot_table(index=["estudante", "escola"], 
                    columns='matéria', 
                    values='nota',
                    margins=True,  # add margins
                    aggfunc='sum')  # sum margins (rows/columns)

,matéria,física,inglês,matemática,All
estudante,escola,,,,
André,Z,50,10,20,80
Bernardo,Y,500,100,200,800
Cristina,Z,5000,1000,2000,8000
Debóra,Y,50000,10000,20000,80000
All,,55550,11110,22220,88880


- É possível também especificar o preenchemento dos valores faltantes 

In [78]:
df_longo.pivot_table(index="estudante", 
                    columns=['escola', 'matéria'], 
                    values='nota', 
                    fill_value=-5)  # replace NaN with -5

escola           Y                           Z                   
matéria     física   inglês matemática  física  inglês matemática
estudante                                                        
André         -5.0     -5.0       -5.0    50.0    10.0       20.0
Bernardo     500.0    100.0      200.0    -5.0    -5.0       -5.0
Cristina      -5.0     -5.0       -5.0  5000.0  1000.0     2000.0
Debóra     50000.0  10000.0    20000.0    -5.0    -5.0       -5.0